In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso

In [2]:
movie1 = pd.read_csv('data/movies_metadata_last12.csv')
movie2 = pd.read_csv('data/movies_metadata_first12.csv')
rate = pd.read_csv('data/ratings_small.csv')
link = pd.read_csv('data/links_small.csv')

In [4]:
movie2 = movie2.drop(columns=
                     ['Unnamed: 0',   
                      'title', 
                      'vote_average' ])
movie1 = movie1.drop(columns=['Unnamed: 0'])

In [5]:
movie2.shape, movie1.shape

((45454, 75), (45451, 310))

In [6]:
movie1.head()

,id,imdb_id,revenue,runtime,vote_average,vote_count,Afghanistan,Albania,Algeria,Angola,...,zu_lan,num_languages,Canceled,In Production,Planned,Post Production,Released,Rumored,tagline_len,video_int
0,862,tt0114709,373554033.0,81.0,7.7,5415.0,0.0,0.0,0.0,0.0,...,0.0,1,0.0,0.0,0.0,0.0,1.0,0.0,0,0
1,8844,tt0113497,262797249.0,104.0,6.9,2413.0,0.0,0.0,0.0,0.0,...,0.0,2,0.0,0.0,0.0,0.0,1.0,0.0,41,0
2,15602,tt0113228,0.0,101.0,6.5,92.0,0.0,0.0,0.0,0.0,...,0.0,1,0.0,0.0,0.0,0.0,1.0,0.0,52,0
3,31357,tt0114885,81452156.0,127.0,6.1,34.0,0.0,0.0,0.0,0.0,...,0.0,1,0.0,0.0,0.0,0.0,1.0,0.0,78,0
4,11862,tt0113041,76578911.0,106.0,5.7,173.0,0.0,0.0,0.0,0.0,...,0.0,1,0.0,0.0,0.0,0.0,1.0,0.0,78,0


In [7]:
movie = pd.merge(movie2, movie1,  how='inner', on=['id', 'imdb_id']).drop_duplicates()
movie.head()

,belongs_to_collection,budget,id,imdb_id,Drama,Crime,Thriller,War,History,Animation,...,zu_lan,num_languages,Canceled,In Production,Planned,Post Production,Released,Rumored,tagline_len,video_int
0,1,3.000000e+07,862,tt0114709,0,0,0,0,0,1,...,0.0,1,0.0,0.0,0.0,0.0,1.0,0.0,0,0
1,0,6.500000e+07,8844,tt0113497,0,0,0,0,0,0,...,0.0,2,0.0,0.0,0.0,0.0,1.0,0.0,41,0
2,1,2.160662e+07,15602,tt0113228,0,0,0,0,0,0,...,0.0,1,0.0,0.0,0.0,0.0,1.0,0.0,52,0
3,0,1.600000e+07,31357,tt0114885,1,0,0,0,0,0,...,0.0,1,0.0,0.0,0.0,0.0,1.0,0.0,78,0
4,1,2.160662e+07,11862,tt0113041,0,0,0,0,0,0,...,0.0,1,0.0,0.0,0.0,0.0,1.0,0.0,78,0


In [8]:
#join with rate's movieId with link's movieId
# join movie's imdb_id with link's imdbID
rate.columns, link.columns

(Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object'),
 Index(['movieId', 'imdbId', 'tmdbId'], dtype='object'))

In [9]:
link = link.rename(columns={"imdbId": "imdb_id"})
link.columns

Index(['movieId', 'imdb_id', 'tmdbId'], dtype='object')

In [10]:
rate.shape, movie.shape, link.shape

((100004, 4), (45423, 383), (9125, 3))

In [11]:
rate_link = pd.merge(rate, link, how="inner", on="movieId")
rate_link.shape

(100004, 6)

In [12]:
mean_rate = rate_link.groupby("imdb_id").mean()
mean_rate.reset_index(level=0, inplace=True)
mean_rate = mean_rate.rename(columns={"rating": "rating_avg"})
mean_rate

,imdb_id,userId,movieId,rating_avg,timestamp,tmdbId
0,417,405.833333,32898.0,4.333333,1.398208e+09,775.0
1,4972,365.000000,7065.0,3.000000,1.365109e+09,618.0
2,6333,468.000000,62383.0,3.500000,1.296190e+09,30266.0
3,8133,558.000000,8511.0,4.250000,1.236383e+09,47653.0
4,9018,511.000000,3309.0,4.250000,1.154476e+09,36208.0
...,...,...,...,...,...,...
9061,5278868,547.000000,151307.0,4.500000,1.472401e+09,373355.0
9062,5480340,98.000000,156025.0,5.000000,1.459405e+09,387893.0
9063,5595168,73.000000,161594.0,3.000000,1.474255e+09,390734.0
9064,5613056,547.000000,160718.0,4.000000,1.469713e+09,399106.0


In [13]:
movie.dropna(subset = ["imdb_id"], inplace=True)
movie.shape

(45406, 383)

In [14]:
movie.imdb_id = [int(x[-7:]) for x in movie.imdb_id]

In [15]:
df = pd.merge(mean_rate, movie, how="inner", on="imdb_id")
print(df.shape)
df.head()

(9010, 388)


,imdb_id,userId,movieId,rating_avg,timestamp,tmdbId,belongs_to_collection,budget,id,Drama,...,zu_lan,num_languages,Canceled,In Production,Planned,Post Production,Released,Rumored,tagline_len,video_int
0,417,405.833333,32898.0,4.333333,1.398208e+09,775.0,0,5.985000e+03,775,0,...,0.0,2,0.0,0.0,0.0,0.0,1.0,0.0,28,0
1,4972,365.000000,7065.0,3.000000,1.365109e+09,618.0,0,1.000000e+05,618,1,...,0.0,1,0.0,0.0,0.0,0.0,1.0,0.0,36,0
2,6333,468.000000,62383.0,3.500000,1.296190e+09,30266.0,0,2.000000e+05,30266,1,...,0.0,1,0.0,0.0,0.0,0.0,1.0,0.0,41,0
3,8133,558.000000,8511.0,4.250000,1.236383e+09,47653.0,0,2.160662e+07,47653,1,...,0.0,2,0.0,0.0,0.0,0.0,1.0,0.0,31,0
4,9018,511.000000,3309.0,4.250000,1.154476e+09,36208.0,0,2.160662e+07,36208,0,...,0.0,1,0.0,0.0,0.0,0.0,1.0,0.0,35,0


In [16]:
print(df.shape)
df = df.drop(columns=['userId', 'movieId', 'imdb_id', 'id', 'tmdbId','timestamp'])
print(df.shape)

(9010, 388)
(9010, 382)


In [17]:
df = df.dropna()
df.to_csv("data/movies_ratings.csv")
df.shape

(8676, 382)

In [21]:
df = df.dropna()
clf = Lasso(alpha=0.001)
X = df.drop(columns=['rating_avg'])
y = df.rating_avg
res = clf.fit(X, y)

coef = clf.coef_

features = X.columns[coef!=0]
print(features)
print(len(X.columns))
print(len(features)) 

Index(['belongs_to_collection', 'budget', 'Drama', 'Crime', 'Thriller',
       'History', 'Animation', 'Comedy', 'Mystery', 'Family', 'Action',
       'Music', 'Documentary', 'Foreign', 'Adventure', 'Horror', 'is_english',
       'Warner Bros.', 'Universal Pictures',
       'Twentieth Century Fox Film Corporation', 'Walt Disney Pictures',
       'New Line Cinema', 'Legendary Pictures', 'Metro-Goldwyn-Mayer (MGM)',
       'Summit Entertainment', 'Canal+', 'Pixar Animation Studios',
       'Miramax Films', 'Working Title Films', 'TriStar Pictures', 'revenue',
       'runtime', 'vote_average', 'vote_count', 'Australia', 'Canada',
       'France', 'Germany', 'Ireland', 'Italy', 'Netherlands', 'Russia',
       'South Africa', 'Switzerland', 'United Kingdom',
       'United States of America', 'num_countries', 'release_date_int',
       'cs_lan', 'de_lan', 'fa_lan', 'fr_lan', 'it_lan', 'ja_lan', 'pt_lan',
       'num_languages', 'tagline_len'],
      dtype='object')
381
57
